# Jupyter Notebook에서 Django 돌리기

In [1]:
import django
import os

SECRET_KEY = "apiserver"
DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.sqlite3',
        'NAME': ':memory:',
    }
}
ROOT_URLCONF = '__main__'

urls = []

os.environ['DJANGO_SETTINGS_MODULE'] = '__main__'

django.setup()

In [2]:
django.get_version()

'2.0.1'

# Form 클래스 사용하기

In [3]:
from django import forms
def odd_validator(value):
    if value % 2 == 0:
        raise forms.ValidationError('짝수라니!!')
        
class QuizForm(forms.Form):
    answer = forms.IntegerField(validators=[odd_validator])

In [4]:
data = {'answer': 10}

In [5]:
form = QuizForm(data)

In [6]:
# answer값이 짝수라서, 유효성 검사 실패
form.is_valid()

False

In [7]:
form.errors

{'answer': ['짝수라니!!']}

# Model 클래스 사용하기

In [8]:
from django.db import models

class Post(models.Model):
    title = models.CharField(max_length=100)
    content = models.TextField()
    created_at = models.DateTimeField(auto_now_add=True)
    updated_at = models.DateTimeField(auto_now=True)
    
    class Meta:
        app_label = 'api'  # 앱이 따로 없으므로, app label을 필히 지정해줘야합니다.
    
    def __str__(self):
        return self.title

jupyter notebook에서 django를 운영하기 때문에 migration을 하지 못하여, 모델 클래스 내역대로 Raw SQL로 DB 테이블을 생성합니다.

In [9]:
from django.db import connection

table_name = Post._meta.db_table

with connection.cursor() as cursor:
    cursor.execute('''
CREATE TABLE "{}"
    ("id" integer NOT NULL PRIMARY KEY AUTOINCREMENT,
    "title" varchar(100) NOT NULL,
    "content" text NOT NULL,
    "created_at" datetime NOT NULL,
    "updated_at" datetime NOT NULL);
'''.format(table_name))

In [10]:
with connection.cursor() as cursor:
    cursor.execute("select name from sqlite_master where type = 'table';")
    for row in cursor.fetchall():
        print(row)

('api_post',)
('sqlite_sequence',)


#### Post 모델을 통해 DB 데이터 추가 

In [11]:
Post.objects.create(
    title="가로수길 애플스토어 오픈 확정… 유니언스퀘어 형태 될 듯",
    content="소문만 무성하던 가로수길 애플 스토어 오픈이 확정됐다. 애플은 한국 애플 스토어 오픈과 관련된 인력 채용에 적극적으로 나서고 있다."
    )

<Post: 가로수길 애플스토어 오픈 확정… 유니언스퀘어 형태 될 듯>

In [12]:
Post.objects.create(
    title="공개SW 공짜 아닌데... 3건 중 1건 라이선스 위반 ",
    content="국내 소프트웨어(SW) 개발 프로젝트에서 공개SW가 사용되는 비율은 90%를 웃돌지만, 3건 중 1건은 라이선스를 위반한 것으로 나타났다.")


<Post: 공개SW 공짜 아닌데... 3건 중 1건 라이선스 위반 >

In [13]:
Post.objects.create(
    title="규제 해커톤 또 무산 위기…카풀업계 속앓이 ",
    content="내달 1~2일 열리는 4차산업혁명위원회 해커톤에서 카풀 애플리케이션 규제 관련 논의가 예정됐지만, 택시업계의 반발로 또 다시 반쪽짜리 토론회가 될 위기다.")

<Post: 규제 해커톤 또 무산 위기…카풀업계 속앓이 >

In [14]:
Post.objects.all()

<QuerySet [<Post: 가로수길 애플스토어 오픈 확정… 유니언스퀘어 형태 될 듯>, <Post: 공개SW 공짜 아닌데... 3건 중 1건 라이선스 위반 >, <Post: 규제 해커톤 또 무산 위기…카풀업계 속앓이 >]>

## ModelForm 사용하기

In [15]:
class PostForm(forms.ModelForm):
    class Meta:
        model = Post
        fields = '__all__'

In [16]:
form = PostForm({'title': 'hello'})

In [17]:
form.is_valid()

False

In [18]:
form.errors

{'content': ['This field is required.']}

In [19]:
for post in Post.objects.all():
    print(post.id, post.title, ':', len(post.content), '글자')

1 가로수길 애플스토어 오픈 확정… 유니언스퀘어 형태 될 듯 : 73 글자
2 공개SW 공짜 아닌데... 3건 중 1건 라이선스 위반  : 77 글자
3 규제 해커톤 또 무산 위기…카풀업계 속앓이  : 86 글자


# DjangoJSONEncoder

In [20]:
import json
from django.core.serializers.json import DjangoJSONEncoder

In [21]:
data = Post.objects.all()
data

<QuerySet [<Post: 가로수길 애플스토어 오픈 확정… 유니언스퀘어 형태 될 듯>, <Post: 공개SW 공짜 아닌데... 3건 중 1건 라이선스 위반 >, <Post: 규제 해커톤 또 무산 위기…카풀업계 속앓이 >]>

TypeError: Object of type 'QuerySet' is not JSON serializable 예외처리

In [22]:
json.dumps(data, cls=DjangoJSONEncoder)

TypeError: <QuerySet [<Post: 가로수길 애플스토어 오픈 확정… 유니언스퀘어 형태 될 듯>, <Post: 공개SW 공짜 아닌데... 3건 중 1건 라이선스 위반 >, <Post: 규제 해커톤 또 무산 위기…카풀업계 속앓이 >]> is not JSON serializable

DjangoJSONEncoder는 QuerySet의 serializer/deserializer 방법을 모르기 때문에 발생한 요류

1. **QuerySet**을 파이썬 표준 테이터타입의 값을 직접 변환

In [23]:
data = [
    {'id': post.id, 'title': post.title, 'content': post.content}
    for post in Post.objects.all()]

json.dumps(data, cls=DjangoJSONEncoder, ensure_ascii=False)

'[{"id": 1, "content": "소문만 무성하던 가로수길 애플 스토어 오픈이 확정됐다. 애플은 한국 애플 스토어 오픈과 관련된 인력 채용에 적극적으로 나서고 있다.", "title": "가로수길 애플스토어 오픈 확정… 유니언스퀘어 형태 될 듯"}, {"id": 2, "content": "국내 소프트웨어(SW) 개발 프로젝트에서 공개SW가 사용되는 비율은 90%를 웃돌지만, 3건 중 1건은 라이선스를 위반한 것으로 나타났다.", "title": "공개SW 공짜 아닌데... 3건 중 1건 라이선스 위반 "}, {"id": 3, "content": "내달 1~2일 열리는 4차산업혁명위원회 해커톤에서 카풀 애플리케이션 규제 관련 논의가 예정됐지만, 택시업계의 반발로 또 다시 반쪽짜리 토론회가 될 위기다.", "title": "규제 해커톤 또 무산 위기…카풀업계 속앓이 "}]'

**ensure_ascii=False** 는 한글 인코딩시 유니코드가 아닌 UTF-8로 인코딩합니다.
api서버에서는 상관없지만 UTF-8로 인코딩하여야 사람이 보기 편함

In [24]:
import json

mydata = ['안녕', '파파파파파파이썬']
json.dumps(mydata)

'["\\uc548\\ub155", "\\ud30c\\ud30c\\ud30c\\ud30c\\ud30c\\ud30c\\uc774\\uc36c"]'

In [25]:
json.dumps(mydata, ensure_ascii=False)

'["안녕", "파파파파파파이썬"]'

.2. json에게 serializer 방법을 알려준다.  
DjangoJSONEncoder가 직렬화 방법을 알고 있기 때문에 이를 확장한다.

    - QuerySet 타입 : tuple 타입으로 변환
    - Post 타입 : dict 타입으로 변환

In [26]:
from django.core.serializers.json import DjangoJSONEncoder
from django.db.models.query import QuerySet

# 커스텀 JSON Encoder를 정의
class MyJSONEncoder(DjangoJSONEncoder):
    def default(self, obj):
        if isinstance(obj, QuerySet):
            return tuple(obj)
        elif isinstance(obj, Post):
            return {'id': obj.id, 'title': obj.title, 'content': obj.content}
        return super().default(obj)

data = Post.objects.all()

# 직렬화할 때, 직렬화를 수행해줄 JSON Encoder를 지정해준다.
json.dumps(data, cls=MyJSONEncoder, ensure_ascii=False)

'[{"id": 1, "content": "소문만 무성하던 가로수길 애플 스토어 오픈이 확정됐다. 애플은 한국 애플 스토어 오픈과 관련된 인력 채용에 적극적으로 나서고 있다.", "title": "가로수길 애플스토어 오픈 확정… 유니언스퀘어 형태 될 듯"}, {"id": 2, "content": "국내 소프트웨어(SW) 개발 프로젝트에서 공개SW가 사용되는 비율은 90%를 웃돌지만, 3건 중 1건은 라이선스를 위반한 것으로 나타났다.", "title": "공개SW 공짜 아닌데... 3건 중 1건 라이선스 위반 "}, {"id": 3, "content": "내달 1~2일 열리는 4차산업혁명위원회 해커톤에서 카풀 애플리케이션 규제 관련 논의가 예정됐지만, 택시업계의 반발로 또 다시 반쪽짜리 토론회가 될 위기다.", "title": "규제 해커톤 또 무산 위기…카풀업계 속앓이 "}]'

## rest_framework.renderer.JSONRender의 직렬화 방식

In [27]:
from rest_framework.renderers import JSONRenderer

data = {'이름': 'Makingfunk'}
json_utf8_string = JSONRenderer().render(data)

json_utf8_string.decode('utf8')

'{"이름":"Makingfunk"}'

JSONRenderer는 Model 타입에 대한 직렬화는 지원하지 않음.

In [28]:
from rest_framework.renderers import JSONRenderer

data = Post.objects.all()

In [29]:
JSONRenderer().render(data)

TypeError: <Post: 가로수길 애플스토어 오픈 확정… 유니언스퀘어 형태 될 듯> is not JSON serializable

In [30]:
data = [
    {'id': post.id, 'title': post.title, 'content': post.content}
    for post in Post.objects.all()]

json_utf8_string = JSONRenderer().render(data)

json_utf8_string.decode('utf8') # 실제 배포에서는 decode를 하지 않음.

'[{"id":1,"content":"소문만 무성하던 가로수길 애플 스토어 오픈이 확정됐다. 애플은 한국 애플 스토어 오픈과 관련된 인력 채용에 적극적으로 나서고 있다.","title":"가로수길 애플스토어 오픈 확정… 유니언스퀘어 형태 될 듯"},{"id":2,"content":"국내 소프트웨어(SW) 개발 프로젝트에서 공개SW가 사용되는 비율은 90%를 웃돌지만, 3건 중 1건은 라이선스를 위반한 것으로 나타났다.","title":"공개SW 공짜 아닌데... 3건 중 1건 라이선스 위반 "},{"id":3,"content":"내달 1~2일 열리는 4차산업혁명위원회 해커톤에서 카풀 애플리케이션 규제 관련 논의가 예정됐지만, 택시업계의 반발로 또 다시 반쪽짜리 토론회가 될 위기다.","title":"규제 해커톤 또 무산 위기…카풀업계 속앓이 "}]'

DRF의 JSONRenderer를 확장할 수도 있지만..

In [31]:
from rest_framework.renderers import JSONRenderer
from rest_framework.utils.encoders import JSONEncoder

class MyJSONEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, Post):
            return {'id': obj.id, 'title': obj.title, 'content': obj.content}
        return super().default(obj)
        
data = Post.objects.all()

renderer = JSONRenderer()
renderer.encoder_class = MyJSONEncoder
json_utf8_string = renderer.render(data)

json_utf8_string.decode('utf8')

'[{"id":1,"content":"소문만 무성하던 가로수길 애플 스토어 오픈이 확정됐다. 애플은 한국 애플 스토어 오픈과 관련된 인력 채용에 적극적으로 나서고 있다.","title":"가로수길 애플스토어 오픈 확정… 유니언스퀘어 형태 될 듯"},{"id":2,"content":"국내 소프트웨어(SW) 개발 프로젝트에서 공개SW가 사용되는 비율은 90%를 웃돌지만, 3건 중 1건은 라이선스를 위반한 것으로 나타났다.","title":"공개SW 공짜 아닌데... 3건 중 1건 라이선스 위반 "},{"id":3,"content":"내달 1~2일 열리는 4차산업혁명위원회 해커톤에서 카풀 애플리케이션 규제 관련 논의가 예정됐지만, 택시업계의 반발로 또 다시 반쪽짜리 토론회가 될 위기다.","title":"규제 해커톤 또 무산 위기…카풀업계 속앓이 "}]'

## ModelSerializer를 통한 JSON 직렬화

DRF에서는 ModelSerializer를 통해 JSONRenders에서 변환가능한 형태로 먼저 데이터를 변환.  

Serializer는 장고의 Form과 유사, ModelSerializer는 장소의 ModelForm과 유사.  

Serializer는 POST 요청만 처리하는 Form이라 할 수 있음

Django Form/ModelForm | Django Serializer/ModelSerializer
---|---
폼 필드 지정 혹은 모델로부터 읽어오기 | 동일
Form HTML을 생성 | JSON 문자열을 생성
입력된 데이터에 대한 유효성 검사 및 획득 | 동일

In [32]:
from rest_framework.serializers import ModelSerializer

# Post모델에 대한 ModelSerializer 정의
class PostModelSerializer(ModelSerializer):
    class Meta:
        model = Post
        fields = '__all__'

In [33]:
post = Post.objects.first()  # Post 타입
post

<Post: 가로수길 애플스토어 오픈 확정… 유니언스퀘어 형태 될 듯>

In [34]:
serializer = PostModelSerializer(post)
serializer.data

ReturnDict([('id', 1),
            ('title', '가로수길 애플스토어 오픈 확정… 유니언스퀘어 형태 될 듯'),
            ('content',
             '소문만 무성하던 가로수길 애플 스토어 오픈이 확정됐다. 애플은 한국 애플 스토어 오픈과 관련된 인력 채용에 적극적으로 나서고 있다.'),
            ('created_at', '2018-01-23T09:03:32.995865'),
            ('updated_at', '2018-01-23T09:03:32.995910')])

serializer.data는 ReturnDict타입. OrderedDict을 상속받았고, 생성자를 통해 serializer필드를 추가호 받음.
(python에서 dict는 순서를 보장하지 않지만. orderedDict는 순서를 보장한다.)

```python
class ReturnDict(OrderedDict):
    def __init__(self, *arg, **kwargs):
        self.serializer = kwargs.pop('serializer')
        super().__init__(*arg, **kwargs)
        # 생략
```

## QuerySet 변환 지원

ModelSerializer는 QuerySet에 대한 변환도 지원. `ModelSerializer`의 `many` 인자는 디폴트 False.  
`many=True` 인자를 지정해야 QuerySet을 처리함.

In [35]:
serializer = PostModelSerializer(Post.objects.all(), many=True)

# 지정된 Model Instance 필드를 통해 list/OrderedDict 획득
serializer.data

[OrderedDict([('id', 1), ('title', '가로수길 애플스토어 오픈 확정… 유니언스퀘어 형태 될 듯'), ('content', '소문만 무성하던 가로수길 애플 스토어 오픈이 확정됐다. 애플은 한국 애플 스토어 오픈과 관련된 인력 채용에 적극적으로 나서고 있다.'), ('created_at', '2018-01-23T09:03:32.995865'), ('updated_at', '2018-01-23T09:03:32.995910')]), OrderedDict([('id', 2), ('title', '공개SW 공짜 아닌데... 3건 중 1건 라이선스 위반 '), ('content', '국내 소프트웨어(SW) 개발 프로젝트에서 공개SW가 사용되는 비율은 90%를 웃돌지만, 3건 중 1건은 라이선스를 위반한 것으로 나타났다.'), ('created_at', '2018-01-23T09:03:33.241359'), ('updated_at', '2018-01-23T09:03:33.241390')]), OrderedDict([('id', 3), ('title', '규제 해커톤 또 무산 위기…카풀업계 속앓이 '), ('content', '내달 1~2일 열리는 4차산업혁명위원회 해커톤에서 카풀 애플리케이션 규제 관련 논의가 예정됐지만, 택시업계의 반발로 또 다시 반쪽짜리 토론회가 될 위기다.'), ('created_at', '2018-01-23T09:03:33.479368'), ('updated_at', '2018-01-23T09:03:33.479400')])]

In [36]:
import json

json.dumps(serializer.data, ensure_ascii=False)

'[{"id": 1, "title": "가로수길 애플스토어 오픈 확정… 유니언스퀘어 형태 될 듯", "content": "소문만 무성하던 가로수길 애플 스토어 오픈이 확정됐다. 애플은 한국 애플 스토어 오픈과 관련된 인력 채용에 적극적으로 나서고 있다.", "created_at": "2018-01-23T09:03:32.995865", "updated_at": "2018-01-23T09:03:32.995910"}, {"id": 2, "title": "공개SW 공짜 아닌데... 3건 중 1건 라이선스 위반 ", "content": "국내 소프트웨어(SW) 개발 프로젝트에서 공개SW가 사용되는 비율은 90%를 웃돌지만, 3건 중 1건은 라이선스를 위반한 것으로 나타났다.", "created_at": "2018-01-23T09:03:33.241359", "updated_at": "2018-01-23T09:03:33.241390"}, {"id": 3, "title": "규제 해커톤 또 무산 위기…카풀업계 속앓이 ", "content": "내달 1~2일 열리는 4차산업혁명위원회 해커톤에서 카풀 애플리케이션 규제 관련 논의가 예정됐지만, 택시업계의 반발로 또 다시 반쪽짜리 토론회가 될 위기다.", "created_at": "2018-01-23T09:03:33.479368", "updated_at": "2018-01-23T09:03:33.479400"}]'

In [37]:
from rest_framework.renderers import JSONRenderer

json_utf8_string = JSONRenderer().render(serializer.data)

json_utf8_string.decode('utf8')

'[{"id":1,"title":"가로수길 애플스토어 오픈 확정… 유니언스퀘어 형태 될 듯","content":"소문만 무성하던 가로수길 애플 스토어 오픈이 확정됐다. 애플은 한국 애플 스토어 오픈과 관련된 인력 채용에 적극적으로 나서고 있다.","created_at":"2018-01-23T09:03:32.995865","updated_at":"2018-01-23T09:03:32.995910"},{"id":2,"title":"공개SW 공짜 아닌데... 3건 중 1건 라이선스 위반 ","content":"국내 소프트웨어(SW) 개발 프로젝트에서 공개SW가 사용되는 비율은 90%를 웃돌지만, 3건 중 1건은 라이선스를 위반한 것으로 나타났다.","created_at":"2018-01-23T09:03:33.241359","updated_at":"2018-01-23T09:03:33.241390"},{"id":3,"title":"규제 해커톤 또 무산 위기…카풀업계 속앓이 ","content":"내달 1~2일 열리는 4차산업혁명위원회 해커톤에서 카풀 애플리케이션 규제 관련 논의가 예정됐지만, 택시업계의 반발로 또 다시 반쪽짜리 토론회가 될 위기다.","created_at":"2018-01-23T09:03:33.479368","updated_at":"2018-01-23T09:03:33.479400"}]'

# 뷰에서의  JSON 응답

### 장고 스타일

JSON 포맷으로 직렬화된 문자열은 장고 뷰를 통해 응답이 이루어져야 함.

1. json.dumps를 통해 직렬화된 문자열을 HttpResponse를 통해 응답
2. json.dumps 기능을 제공하는 JsonResponse를 사용

JsonResponse는 장고의 DjangoJSONEncoder를 사용하기 때문데, QuerySet에 대한 직렬화는 불가능.  

In [38]:
# 직렬화할 QuerySet 준비
data = Post.objects.all()
data

<QuerySet [<Post: 가로수길 애플스토어 오픈 확정… 유니언스퀘어 형태 될 듯>, <Post: 공개SW 공짜 아닌데... 3건 중 1건 라이선스 위반 >, <Post: 규제 해커톤 또 무산 위기…카풀업계 속앓이 >]>

JsonResponse에 넘겨둘 인자를 준비

- encoder(디폴트:DjangoJSONEncoder): JSON 인코딩을 수행할 클래스
- safe(디폴트:True): 변환할 데이터의 dict 타입 체킹이 목적. 데이터가 dict타입이 아니면 False로 지정해야함. 미지정시 TypeError  발생.
- json_dumps_params(디폴트:None): json.dumps에 넘겨질 인자
- kwargs(디폴트: {}): 부모 클래스인 HttpResponse에 넘겨질 인자

In [39]:
encoder = MyJSONEncoder
safe = False  # True : data가 dict / False : data가 dict가 아닐 경우
json_dumps_params = {'ensure_ascii': False}
kwargs = {}  # HttpResponse에 전해지는 Keyword 인자

Http 응답을 생성하고, 그 응답바디를 출력

In [40]:
from django.http import JsonResponse

response = JsonResponse(data, encoder, safe, json_dumps_params, **kwargs)

print(response)
response.content.decode('utf8')

<JsonResponse status_code=200, "application/json">


'[{"id": 1, "content": "소문만 무성하던 가로수길 애플 스토어 오픈이 확정됐다. 애플은 한국 애플 스토어 오픈과 관련된 인력 채용에 적극적으로 나서고 있다.", "title": "가로수길 애플스토어 오픈 확정… 유니언스퀘어 형태 될 듯"}, {"id": 2, "content": "국내 소프트웨어(SW) 개발 프로젝트에서 공개SW가 사용되는 비율은 90%를 웃돌지만, 3건 중 1건은 라이선스를 위반한 것으로 나타났다.", "title": "공개SW 공짜 아닌데... 3건 중 1건 라이선스 위반 "}, {"id": 3, "content": "내달 1~2일 열리는 4차산업혁명위원회 해커톤에서 카풀 애플리케이션 규제 관련 논의가 예정됐지만, 택시업계의 반발로 또 다시 반쪽짜리 토론회가 될 위기다.", "title": "규제 해커톤 또 무산 위기…카풀업계 속앓이 "}]'

### django-rest-framework 스타일

In [41]:
# 변환할 데이터로서 QuerySet 준비
queryset = Post.objects.all()

In [42]:
# queryset을 통해 ModelSerializer 준비
serializer = PostModelSerializer(queryset, many=True)
serializer

PostModelSerializer(<QuerySet [<Post: 가로수길 애플스토어 오픈 확정… 유니언스퀘어 형태 될 듯>, <Post: 공개SW 공짜 아닌데... 3건 중 1건 라이선스 위반 >, <Post: 규제 해커톤 또 무산 위기…카풀업계 속앓이 >]>, many=True):
    id = IntegerField(label='ID', read_only=True)
    title = CharField(max_length=100)
    content = CharField(style={'base_template': 'textarea.html'})
    created_at = DateTimeField(read_only=True)
    updated_at = DateTimeField(read_only=True)

In [43]:
serializer.data

[OrderedDict([('id', 1), ('title', '가로수길 애플스토어 오픈 확정… 유니언스퀘어 형태 될 듯'), ('content', '소문만 무성하던 가로수길 애플 스토어 오픈이 확정됐다. 애플은 한국 애플 스토어 오픈과 관련된 인력 채용에 적극적으로 나서고 있다.'), ('created_at', '2018-01-23T09:03:32.995865'), ('updated_at', '2018-01-23T09:03:32.995910')]), OrderedDict([('id', 2), ('title', '공개SW 공짜 아닌데... 3건 중 1건 라이선스 위반 '), ('content', '국내 소프트웨어(SW) 개발 프로젝트에서 공개SW가 사용되는 비율은 90%를 웃돌지만, 3건 중 1건은 라이선스를 위반한 것으로 나타났다.'), ('created_at', '2018-01-23T09:03:33.241359'), ('updated_at', '2018-01-23T09:03:33.241390')]), OrderedDict([('id', 3), ('title', '규제 해커톤 또 무산 위기…카풀업계 속앓이 '), ('content', '내달 1~2일 열리는 4차산업혁명위원회 해커톤에서 카풀 애플리케이션 규제 관련 논의가 예정됐지만, 택시업계의 반발로 또 다시 반쪽짜리 토론회가 될 위기다.'), ('created_at', '2018-01-23T09:03:33.479368'), ('updated_at', '2018-01-23T09:03:33.479400')])]

뷰에서는 Response를 통해 응답을 생성. 이는 HttpResponse를 상속받은 클래스임.  
Response는 단순히 JSON 직렬화뿐만 아니라, HTTP 요청에 따라 다양한 포맷으로 변환(rener)하여 응답을 생성할 수 있음

In [44]:
from rest_framework.response import Response

response = Response(serializer.data)
response

<Response status_code=200, "text/html; charset=utf-8">

Response 객체에 변환에 필요한 속성을 지정해야 함.  
실제 요청을 처리하는 코드에서는 APIView 클래스에 의해서 디폴트 지정되므로, 일반적으로 직접 지정하는 일은 없음

In [45]:
from rest_framework.views import APIView

renderer_cls = APIView.renderer_classes[0]
renderer_obj = renderer_cls()
response.accepted_renderer = renderer_obj   # JSON 변환을 위한 JSONRenderer 인스턴스

response.accepted_media_type = renderer_obj.media_type  # 'application/json
response.renderer_context = {'view': None, 'args': (), 'kwargs': {}, 'request': None}

In [46]:
response

<Response status_code=200, "text/html; charset=utf-8">

response 객체는 아직 변환할 준비만 하고 있을 뿐, 아직 JSON 직렬화 변환을 수행하지 않음.  
.rendered_content 속성에 접근할 때, 변환이 이루어짐

In [47]:
response.rendered_content.decode('utf8')

'[{"id":1,"title":"가로수길 애플스토어 오픈 확정… 유니언스퀘어 형태 될 듯","content":"소문만 무성하던 가로수길 애플 스토어 오픈이 확정됐다. 애플은 한국 애플 스토어 오픈과 관련된 인력 채용에 적극적으로 나서고 있다.","created_at":"2018-01-23T09:03:32.995865","updated_at":"2018-01-23T09:03:32.995910"},{"id":2,"title":"공개SW 공짜 아닌데... 3건 중 1건 라이선스 위반 ","content":"국내 소프트웨어(SW) 개발 프로젝트에서 공개SW가 사용되는 비율은 90%를 웃돌지만, 3건 중 1건은 라이선스를 위반한 것으로 나타났다.","created_at":"2018-01-23T09:03:33.241359","updated_at":"2018-01-23T09:03:33.241390"},{"id":3,"title":"규제 해커톤 또 무산 위기…카풀업계 속앓이 ","content":"내달 1~2일 열리는 4차산업혁명위원회 해커톤에서 카풀 애플리케이션 규제 관련 논의가 예정됐지만, 택시업계의 반발로 또 다시 반쪽짜리 토론회가 될 위기다.","created_at":"2018-01-23T09:03:33.479368","updated_at":"2018-01-23T09:03:33.479400"}]'

### 실전에서의 Response 활용

실전에서는 간결하게 사용함

In [48]:
from rest_framework import generics

class PostListAPIView(generics.ListAPIView):
    queryseet = Post.objects.all()
    serailizer_class = PostModelSerializer

간결하게 정의한 뷰 만으로 다음과 같은 JSON 응답을 만들 수 있음

In [49]:
from django.http import HttpRequest

class DummyUser:
    pass

request = HttpRequest()
request.user = DummyUser()
request.method = 'GET'

response = PostListAPIView.as_view()(request)
response.rendered_content.decode('utf8')

WrappedAttributeError: 'DummyUser' object has no attribute 'is_active'